# Random Forest Algorithm

#### Importación de librerías ⬇️

In [2]:
import pickle
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import export_text, plot_tree
from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, f1_score, confusion_matrix, root_mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import seaborn as sns

In [3]:
# Importar data de un csv
data = pd.read_csv('los_data_gt.csv')

In [4]:
data.columns

Index(['dias_estancia', 'sexo', 'grupo_etnico', 'departamento', 'municipio',
       'causa_atencion', 'condicion_egreso', 'tratamiento_recibido',
       'edad_correcta', 'causa_categoria', 'edad_categoria', 'region'],
      dtype='object')

In [5]:
data.head()

,dias_estancia,sexo,grupo_etnico,departamento,municipio,causa_atencion,condicion_egreso,tratamiento_recibido,edad_correcta,causa_categoria,edad_categoria,region
0,1.0,Hombre,Ignorado,Guatemala,San Juan Sacatepéquez,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Central
1,1.0,Mujer,Ignorado,Zacapa,Zacapa,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Nororiente
2,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente
3,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente
4,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente


In [5]:
dr = data.copy()

In [6]:
# Evaluación de variables
def change(variable, dictValue, df2):
    count = 1

    def changeToVariable(x):
        nonlocal count
        nonlocal dictValue
        if x in dictValue:
            return dictValue[x]
        else:
            dictValue[x] = count
            count += 1
            return dictValue[x]
        
    if not pd.api.types.is_numeric_dtype(df2[variable]):
        df2[variable] = df2[variable].apply(changeToVariable)
    
        print(f'Para la variable {variable} los valores ahora son: ')
        for key, value in dictValue.items():
            print(key, ":", value)
    else:
        print(f"La columna {variable} es numerica")


def chageNotNumericVars(df):
    dicVars = {}
    for col in df.columns:
        change(col, dicVars, df)
    return dicVars

In [7]:
y = dr.pop('dias_estancia')
dr.pop('condicion_egreso')
x = dr
#description = chageNotNumericVars(dr)

In [8]:
# Separar los conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (1782662, 10)
Tamaño del conjunto de prueba: (445666, 10)


## Primer modelo: Regression Random Forest
pop condicion_egreso

In [6]:
def regressionRandomForest(X_train, y_train, X_test, y_test, deep=None):

    # Crear el modelo de Random Forest
    rf_model = RandomForestRegressor(random_state=42)

    rf_model.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = rf_model.predict(X_test)

    # Calcular el error cuadrático medio
    mse = mean_squared_error(y_test, y_pred)
    print("Error cuadrático medio:", mse)

    # Calcular la raiz del error cuadratico medio
    rmse = root_mean_squared_error(y_test, y_pred)
    print("raiz del error cuadratico medio", rmse)

    # Calcular el coeficiente de determinación
    r2 = r2_score(y_test, y_pred)
    print("Coeficiente de determinación:", r2)

In [11]:
regressionRandomForest(X_train, y_train, X_test, y_test)

Error cuadrático medio: 6.637661921527838
raiz del error cuadratico medio 2.576366030192107
Coeficiente de determinación: -0.00632898080276334


In [7]:
def findBestMaxDepth(X_train, y_train):
    # Crear el modelo de Random Forest
    rf_model = RandomForestRegressor(random_state=42)

    # Definir los hiperparámetros
    param_grid = {
        'max_depth': list(range(5, 25, 5))
    }

    # Crear el objeto de búsqueda en la grilla
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

    # Realizar la búsqueda en la grilla
    grid_search.fit(X_train, y_train)

    # Obtener la mejor profundidad
    best_depth = grid_search.best_params_['max_depth']

    return best_depth

In [17]:
best_depth = findBestMaxDepth(X_train, y_train)
print(f"La mejor profundidad encontrada es: {best_depth}")

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ........................................max_depth=5; total time= 2.2min
[CV] END ........................................max_depth=5; total time= 2.2min
[CV] END ........................................max_depth=5; total time= 2.2min
[CV] END .......................................max_depth=10; total time= 4.0min
[CV] END .......................................max_depth=10; total time= 4.0min
[CV] END .......................................max_depth=10; total time= 4.0min
[CV] END .......................................max_depth=15; total time= 5.1min
[CV] END .......................................max_depth=15; total time= 5.1min
[CV] END .......................................max_depth=15; total time= 4.4min
[CV] END .......................................max_depth=20; total time= 5.0min
[CV] END .......................................max_depth=20; total time= 5.0min
[CV] END .......................................m

In [23]:
# Crear el modelo de Random Forest con la mejor profundidad
rf_model = RandomForestRegressor(max_depth=best_depth, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

# Calcular la raiz del error cuadratico medio
rmse = root_mean_squared_error(y_test, y_pred)
print("raiz del error cuadratico medio", rmse)

# Calcular el coeficiente de determinación
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación:", r2)

Error cuadrático medio: 5.841146730337402
raiz del error cuadratico medio 2.4168464432680454
Coeficiente de determinación: 0.11442985416364448


## Segundo modelo: Regression Random Forest
pop etnia=ignorado, causa_atencion, edad_correcta

In [8]:
df_cp = data.copy()

df_cp = df_cp.loc[df_cp['grupo_etnico'] != 'Ignorado']

df_cp.pop('condicion_egreso')
df_cp.pop('causa_atencion')
df_cp.pop('edad_correcta')

df_cp = pd.get_dummies(df_cp)

y = df_cp.pop('dias_estancia')
X = df_cp

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
best_depth = findBestMaxDepth(X_train, y_train)
print(f"La mejor profundidad encontrada es: {best_depth}")

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ........................................max_depth=5; total time=25.6min
[CV] END ........................................max_depth=5; total time=25.6min
[CV] END ........................................max_depth=5; total time=25.8min
[CV] END .......................................max_depth=10; total time=51.6min
[CV] END .......................................max_depth=10; total time=51.8min
[CV] END .......................................max_depth=10; total time=67.5min
[CV] END .......................................max_depth=15; total time=88.0min
[CV] END .......................................max_depth=15; total time=88.1min
[CV] END .......................................max_depth=15; total time=85.7min
[CV] END .......................................max_depth=20; total time=92.0min
[CV] END .......................................max_depth=20; total time=92.1min
[CV] END .......................................m

### Test Data Set

In [10]:
# Crear el modelo de Random Forest con la mejor profundidad
rf_model = RandomForestRegressor(max_depth=best_depth, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

# Calcular la raiz del error cuadratico medio
rmse = root_mean_squared_error(y_test, y_pred)
print("raiz del error cuadratico medio", rmse)

# Calcular el coeficiente de determinación
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación:", r2)

Error cuadrático medio: 6.476403143525614
raiz del error cuadratico medio 2.544877824872073
Coeficiente de determinación: 0.0794292018639483


### Train Data Set

In [11]:
# Crear el modelo de Random Forest con la mejor profundidad
rf_model = RandomForestRegressor(max_depth=best_depth, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de entrenamiento
y_train_pred = rf_model.predict(X_train)

# Calcular el error cuadrático medio
mse_train = mean_squared_error(y_train, y_train_pred)
print("Error cuadrático medio (entrenamiento):", mse_train)

# Calcular la raiz del error cuadratico medio
rmse_train = np.sqrt(mse_train)
print("Raiz del error cuadratico medio (entrenamiento):", rmse_train)

# Calcular el coeficiente de determinación
r2_train = r2_score(y_train, y_train_pred)
print("Coeficiente de determinación (entrenamiento):", r2_train)

Error cuadrático medio (entrenamiento): 6.15908768059046
Raiz del error cuadratico medio (entrenamiento): 2.4817509304099112
Coeficiente de determinación (entrenamiento): 0.11452255212285312
